## Merge compounds

In [ ]:
suffix = ''  # suffix for analysis
hit_filename = 'hits.filtered.sdf'
pdb_filename = 'reference.pdb'
ranking = '∆∆G' #@param ["LE", "∆∆G", "comRMSD"]
joining_cutoff:int = 5
quick_reananimation:bool = False
output_folder = 'frag_output'
sw_dist = 25
sw_length = 50

In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem, Draw, PandasTools, BRICS
from rdkit.Chem.Draw import IPythonConsole
import pandas as pd
import pandera.typing as pdt
from typing import List, Dict

with Chem.SDMolSupplier(hit_filename) as sd:
    hits: Dict[str, Chem.Mol] = {mol.GetProp('_Name'): mol for mol in sd}
    
    
with open(pdb_filename) as fh:
    pdbblock = fh.read()
    
# ------------------------------------------------------
    
import logging
import pyrosetta_help as ph

logger = ph.configure_logger()
logger.handlers[0].setLevel(logging.ERROR)  # logging.WARNING = 30

from fragmenstein import Igor

Igor.init_pyrosetta()
# # Equivalent to:
# import pyrosetta
# extra_options = ph.make_option_string(no_optH=False,
#                                       ex1=None,
#                                       ex2=None,
#                                       #mute='all',
#                                       ignore_unrecognized_res=True,
#                                       load_PDB_components=False,
#                                       ignore_waters=True)
# pyrosetta.init(extra_options=extra_options)

In [ ]:
import os
if not os.path.exists(output_folder):
    # isnt this done automatically?
    os.mkdir(output_folder)

import os, re
import pyrosetta, logging
import pandas as pd
from rdkit import Chem
from fragmenstein import Victor, Laboratory

Victor.work_path = output_folder
Victor.monster_throw_on_discard = True  # stop this merger if a fragment cannot be used.
Victor.monster_joining_cutoff = joining_cutoff  # Å
Victor.quick_reanimation = quick_reananimation  # for the impatient
Victor.error_to_catch = Exception  # stop the whole laboratory otherwise
#Victor.enable_stdout(logging.ERROR)
Victor.enable_logfile(os.path.join(output_folder, 'fragmenstein.log'), logging.ERROR)

In [ ]:
import random, time
from rdkit import Chem, rdBase

# calculate!
lab = Laboratory(pdbblock=pdbblock, covalent_resi=None)
n_cores = 55  #@param {type:"integer"}
hitlist = list(hits.values())

tick = time.time()
combinations:pd.DataFrame = lab.combine(hitlist, n_cores=n_cores)
with rdBase.BlockLogs():
    combinations['simple_smiles'] = combinations.unminimized_mol.apply(Victor.to_simple_smiles)
combinations.to_pickle(f'fragmenstein_mergers{suffix}.pkl.gz')
combinations.to_csv(f'fragmenstein_mergers{suffix}.csv')
print(tick - time.time())

In [ ]:
import plotly.express as px

px.histogram(combinations.loc[(combinations['∆∆G'] < 0) & (combinations.outcome == 'acceptable')], '∆∆G', title='Distribution of ∆∆G')

In [ ]:
from smallworld_api import SmallWorld
from warnings import warn

sws = SmallWorld()

chemical_databases:pd.DataFrame = sws.retrieve_databases()

queries = combinations.sort_values(ranking).loc[(combinations.outcome == 'acceptable')].drop_duplicates('simple_smiles').reset_index().head(500)

similars = sws.search_many(queries.simple_smiles.to_list(),
                           dist=sw_dist,
                           length=sw_length,
                           db=sws.REAL_dataset,
                           tolerated_exceptions=Exception)
# query_index was added clientside to keep track!
similars['query_name'] = similars.query_index.map( queries.name.to_dict() )
similars['hits'] = similars.query_index.map( queries.hit_mols.to_dict() )
similars['minimized_merger'] = similars.query_index.map( queries.minimized_mol.to_dict() )
similars['unminimized_merger'] = similars.query_index.map( queries.unminimized_mol.to_dict() )
similars['name'] = similars['id'] + ':' + similars['query_name']
similars['smiles'] = similars.hitSmiles.str.split(expand=True)[0]
similars.to_pickle(f'similars{suffix}.pkl.gz')
print(f'Found {len(similars)} analogues')

In [ ]:
from typing import Dict

def get_custom_map(row: pd.Series) -> Dict[str, Dict[int, int]]:
    """
    SmallWorld returns a mapping of the indices.
    This functions maps the indices back to the original hit molecules.
    :param row:
    :return:
    """
    temp = Victor(row.hits, pdb_block=pdbblock)
    temp.monster.positioned_mol = row.unminimized_merger
    temp.minimized_mol = row.minimized_merger
    return temp.migrate_sw_origins(row)

similars['custom_map'] = similars.apply(get_custom_map, axis=1)

In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem, Draw, PandasTools, BRICS
from rdkit.Chem.Draw import IPythonConsole
import pandas as pd
import pandera.typing as pdt
from typing import List, Dict
    
# ------------------------------------------------------
    
import logging
import pyrosetta_help as ph
import pyrosetta

ranking = '∆∆G' #@param ["LE", "∆∆G", "comRMSD"]
joining_cutoff:int = 5
quick_reananimation:bool = False
output_folder = 'frag_output'

import os
if not os.path.exists(output_folder):
    # isnt this done automatically?
    os.mkdir(output_folder)

import os, re
import pyrosetta, logging
import pandas as pd
from rdkit import Chem
from fragmenstein import Victor, Laboratory

Victor.work_path = output_folder
Victor.monster_throw_on_discard = False  # unmin + hits combo
Victor.monster_joining_cutoff = joining_cutoff  # Å
Victor.quick_reanimation = quick_reananimation  # for the impatient
Victor.error_to_catch = Exception  # stop the whole laboratory otherwise
#Victor.enable_stdout(logging.ERROR)
Victor.enable_logfile(os.path.join(output_folder, 'fragmenstein.log'), logging.ERROR)

from fragmenstein.laboratory.validator import place_input_validator

n_cores = 55

lab = Laboratory(pdbblock=pdbblock, covalent_resi=None)
placements:pd.DataFrame = lab.place(place_input_validator(similars), n_cores=n_cores)
placements.loc[(placements['∆∆G'] > -1) & (placements.outcome == 'acceptable'), 'outcome'] = 'weak'
placements['unminimized_mol'] = placements.unminimized_mol.fillna(Chem.Mol())
placements.to_pickle(f'placed{suffix}.pkl.gz')
placements.to_csv(f'placed{suffix}.csv')
placements.outcome.value_counts()

In [ ]:
import plotly.express as px

px.histogram(placements.loc[(placements['∆∆G'] < 0) & (placements.outcome == 'acceptable')], '∆∆G', title='Distribution of ∆∆G')

In [ ]:
from pathlib import Path
import re, operator

def split_name(name):
    if not name:
        return ('','')
    if 'PV-' in name:
        match = re.match(r'(PV-\d+)-(.*)-(.*)', name).groups()
    else:
        match = re.match(r'(Z\d+)-(.*)-(.*)', name).groups()
    return (match[0], f"{match[1]}-{match[2]}" )

names = placements['name'].apply(split_name)

placements['enamine_name'] = names.apply(operator.itemgetter(0))
placements['merger_name'] = names.apply(operator.itemgetter(1))

# -----------------

n2mol = combinations.minimized_mol.to_dict()
# there seems to be some name bleaching? if so:
#n2mol = dict(zip([n.replace('_', '-') for n in n2mol.keys()], n2mol.values()))
placements['merger_mol'] = placements.merger_name.map(n2mol)

placements['hit_names'] = placements.merger_name.str.split('-')
placements['LE'] = placements.LE.abs()
placements['path'] = placements.name.apply(lambda n: Path('frag_output') / n / f'{n}.holo_minimised.pdb')

placements.to_pickle(f'placed{suffix}.pkl.gz')
placements.to_csv(f'placed{suffix}.csv')

In [ ]:
valids = placements.loc[placements.outcome == 'acceptable'].sort_values('∆∆G')
valids['combined_name'] = valids['name']
valids['name'] = valids['enamine_name']

valids.to_pickle(f'acceptables{suffix}.pkl.gz')

valids.path.apply(Path.exists).value_counts()

In [ ]:
"""
https://github.com/matteoferla/PLIP-PyRosetta-hotspots-test/blob/main/plipspots_docking/plipspots/serial.py

This is a class I use to _apply_ PLIP to a pd.Series of molecules.
It is not built for the project, but works.
Note I have not dumped the methods that are not needed for the project.
"""

import os
from pathlib import Path
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem import PandasTools

from functools import singledispatchmethod
from typing import Tuple, Dict, List, Union
from collections import Counter, defaultdict
from plip.structure.preparation import PDBComplex, PLInteraction
from openbabel.pybel import Atom, Residue
from openbabel.pybel import ob
from fragmenstein.victor import MinimalPDBParser
import warnings


class SerialPLIPper:
    """
    Calling the instance will return a ``Dict[Tuple[str, str, int], int]``,
    where the key is interaction type, residue 3-letter name, residue index
    and the value is the count of interactions.
    Basically, applying Plip to a pd.Series of Chem.Mol.

    Unplacking it is kind of wierd, the best way I reckon is a brutal for-loop:

    .. code-block:: python

        import pandas as pd
        import pandera.typing as pdt

        intxndexes: pdt.Series[Dict[Tuple[str, str, int], int]] = hits.ROMol.apply(SerialPLIPper(pdb_filename))
        # columns will still be a tuple...:
        intxn_df = pd.DataFrame(intxndexes.to_list()).fillna(0).astype(int)
        hits['N_interactions'] = intxn_df.sum(axis='columns')
        for c in sorted(intxn_df.columns, key=lambda kv: kv[2]):
            # columns will be a colon-separated string:
            hits[':'.join(map(str, c))] = intxn_df[c]
    """

    def __init__(self, pdb_block: str, resn='LIG', chain='B'):
        assert 'ATOM' in pdb_block, f'No ATOM entry in block provided: {pdb_block}'
        self.pdb_block = pdb_block
        self.resn = resn
        self.chain = chain

    @classmethod
    def from_filename(cls, pdb_filename: str, *args, **kwargs):
        """
        The main constructor is from PDB block, this is from PDB file
        """
        with open(pdb_filename, 'r') as f:
            pdb_block = f.read()
        return cls(pdb_block, *args, **kwargs)

    def __call__(self, mol) -> Dict[Tuple[str, str, int], int]:
        if mol is None or not isinstance(mol, Chem.Mol) or mol.GetNumAtoms() == 0:
            return {}
        holo: str = self.plonk(mol)
        interaction_set: PLInteraction = self.get_interaction_set(holo)
        return self.get_interaction_counts(interaction_set)

    def assign_pdb(self, mol: Chem.Mol):
        """
        Fix the PDB info for the molecule, in place
        """
        counts = defaultdict(int)
        atom: Chem.Atom
        for atom in mol.GetAtoms():
            element: str = atom.GetSymbol()
            counts[element] += 1
            info = Chem.AtomPDBResidueInfo(atomName=f'{element: >2}{counts[element]: <2}',
                                           residueName=self.resn,
                                           residueNumber=1, chainId=self.chain)
            atom.SetPDBResidueInfo(info)

    def plonk(self, mol):
        """
        Temporarily here. Do not copy.
        There likely is a way to do this in OBabel
        This is using Fragmenstein ``MinimalPDBParser``.

        :param mol:
        :return:
        """
        pdbdata = MinimalPDBParser(self.pdb_block, remove_other_hetatms=True, ligname=self.resn)
        self.assign_pdb(mol)
        moldata = MinimalPDBParser(Chem.MolToPDBBlock(mol))
        pdbdata.append(moldata)
        return str(pdbdata)

    @singledispatchmethod
    def get_interaction_set(self) -> PLInteraction:
        """
        Overloaded method: block or mol return the iternaction set
        :return:
        """
        raise NotImplementedError

    @get_interaction_set.register
    def _(self, block: str) -> PLInteraction:
        holo = PDBComplex()
        holo.load_pdb(block, as_string=True)
        holo.analyze()
        return holo.interaction_sets[':'.join([self.resn, self.chain, str(1)])]

    @get_interaction_set.register
    def _(self, mol: Chem.Mol) -> PLInteraction:
        if mol.GetNumAtoms() == 0:
            raise ValueError('Molecule has no atoms')
        holo = PDBComplex()
        holo.load_pdb(self.plonk(mol), as_string=True)
        holo.analyze()
        return holo.interaction_sets[':'.join([self.resn, self.chain, str(1)])]

    def get_atomname(self, atom: Union[Atom, ob.OBAtom]) -> str:
        """
        Given an atom, return its name.
        """
        if isinstance(atom, Atom):
            res: ob.OBResidue = atom.residue.OBResidue
            obatom = atom.OBAtom
        elif isinstance(atom, ob.OBAtom):
            obatom: ob.OBAtom = atom
            res: ob.OBResidue = obatom.GetResidue()
        else:
            raise TypeError
        return res.GetAtomID(obatom)

    def get_atom_by_atomname(self, residue: Union[ob.OBResidue, Residue], atomname: str) -> ob.OBAtom:
        """
        Get an atom by its name in a residue.
        """
        if isinstance(residue, Residue):
            residue = residue.OBResidue
        obatom: ob.OBAtom
        for obatom in ob.OBResidueAtomIter(residue):
            if residue.GetAtomID(obatom).strip() == atomname:
                return obatom
        else:
            raise ValueError(f'No atom with name {atomname} in residue {residue.GetName()}')

    def get_interaction_counts(self, interaction_set: PLInteraction) -> Dict[Tuple[str, str, int], int]:
        """
        Count the number of interactions of each type for each residue
        """
        intxns: List = interaction_set.all_itypes
        intxn_dex = defaultdict(int)
        for intxn in intxns:
            key = (intxn.__class__.__name__, intxn.restype, intxn.resnr)
            intxn_dex[key] += 1
        return dict(sorted(intxn_dex.items(), key=lambda kv: kv[0][2]))

In [ ]:
chain_check = {l[21:26].strip() for l in valids.iloc[0].path.read_text().split('\n') if ' LIG ' in l} - {'LIG'}
print(chain_check)
chain = next(iter(chain_check)).split()[0]

import operator

plipper = SerialPLIPper('ATOM', resn='LIG', chain=chain)
def get_intxns(path: Path) -> dict:
    interaction_set: PLInteraction = plipper.get_interaction_set(path.read_text())
    return plipper.get_interaction_counts(interaction_set)

intxns = valids.path.apply(get_intxns)
key_order = sorted(set(intxns.apply(lambda d: list(d.keys())).sum()), key=operator.itemgetter(2))
for key in key_order:
    valids[key] = intxns.apply(lambda d: d.get(key, 0))
valids['N_interactions'] = valids[key_order].sum(axis=1)

valids.to_pickle(f'acceptables{suffix}.pkl.gz')
valids[['name', 'regarded', 'smiles', '∆∆G', '∆G_bound', '∆G_unbound',
       'comRMSD', 'N_constrained_atoms', 'N_unconstrained_atoms', 'runtime',
       'LE',  'outcome',
       'percent_hybrid', 'N_interactions'] + key_order].to_csv(f'interactions{suffix}.csv')
print('done')

In [ ]:
valids.sort_values('∆∆G').head(20)[['∆∆G', 'name','merger_name', 'comRMSD', 'N_constrained_atoms', 'N_unconstrained_atoms', 'N_interactions']]

In [ ]:
from rdkit.Chem import PandasTools

short = combinations.loc[(combinations.outcome == 'acceptable')].sort_values('∆∆G').reset_index().drop_duplicates('name')

PandasTools.WriteSDF(df=short,
                    out=f'combinations{suffix}.sdf',
                    molColName='minimized_mol',
                    idName='name',
                    properties=['regarded', 'smiles', '∆∆G', '∆G_bound', '∆G_unbound',
       'comRMSD', 'N_constrained_atoms', 'N_unconstrained_atoms', 'runtime',
       'LE',  'outcome',
       'percent_hybrid']
                    )

short[['name', 'regarded', 'smiles', '∆∆G', '∆G_bound', '∆G_unbound',
       'comRMSD', 'N_constrained_atoms', 'N_unconstrained_atoms', 'runtime',
       'LE',  'outcome',
       'percent_hybrid']].to_csv(f'combinations{suffix}.csv')

In [ ]:
from rdkit.Chem import PandasTools

placements['regarded'] = placements.merger_name.str.split('-')
short = placements.loc[(placements.outcome == 'acceptable')].sort_values('∆∆G').drop_duplicates('name')

PandasTools.WriteSDF(df=short,
                    out=f'placements{suffix}.sdf',
                    molColName='minimized_mol',
                    idName='name',
                    properties=['regarded', 'merger_name', 'smiles', '∆∆G', '∆G_bound', '∆G_unbound',
       'comRMSD', 'N_constrained_atoms', 'N_unconstrained_atoms', 'runtime',
       'LE',  'outcome',
       'percent_hybrid']
                    )

short[['name', 'regarded', 'smiles', '∆∆G', '∆G_bound', '∆G_unbound',
       'comRMSD', 'N_constrained_atoms', 'N_unconstrained_atoms', 'runtime',
       'LE',  'outcome',
       'percent_hybrid']].to_csv(f'placements{suffix}.csv')

In [ ]:
valids['merger_mol'] = valids.merger_name.map(placements.set_index('merger_name').minimized_mol.to_dict().get)

In [ ]:
valids.hit_names

In [ ]:
valids = valids.loc[~valids.merger_mol.isna()]

In [ ]:
# def unbleach(names: list, fluff_marker = '§'):
#     return [f'{names[0]}_{names[1]}{fluff_marker}{names[2]}',
#             f'{names[3]}_{names[4]}{fluff_marker}{names[5]}',
#            ]

def unbleach(names: list, fluff_marker = '§'):
    if len(names) < 3:
        return names
    if len(names) == 6:
        return [f'{names[0]}_{names[1]}{fluff_marker}{names[2]}',
                f'{names[3]}_{names[4]}{fluff_marker}{names[5]}',
               ]
    if len(names[1]) < 3:
        return [f'{names[0]}_{names[1]}{fluff_marker}{names[2]}',
                f'{names[3]}',
               ]
    if len(names[2]) < 3:
        return [f'{names[0]}',
                f'{names[1]}_{names[2]}{fluff_marker}{names[3]}'
               ]
    else:
        raise Exception(names)
        
valids['hit_names'] = valids.hit_names.apply(unbleach)

In [ ]:
from michelanglo_api import MikeAPI
from rdkit import Chem
from typing import List
import pandas as pd
import json

# declare variables
hit_filename = 'EV-D68_hits.sdf'
hit_names = list(hits.keys())
with Chem.SDWriter(str(Path('upload') / hit_filename)) as sdfh:
    for name, hit in hits.items():
        hit.SetProp('_Name', name)
        sdfh.write(hit)
        
headers = ['name', 'hits', 'RMSD', '∆∆G', 'LE', 'N unconstrained atoms', 'N constrained atoms', 'N interactions', 'SMILES']
metadata: pd.DataFrame = valids.loc[~valids.merger_mol.isna()]\
                               .rename(columns=dict(hit_names='hits', 
                                                    comRMSD='RMSD',
                                                    dist='distance',
                                                    smiles='SMILES',
                                                    N_constrained_atoms='N constrained atoms',
                                                    N_unconstrained_atoms='N unconstrained atoms',
                                                    N_interactions='N interactions'))[headers]
metadata_filename: str = 'EV-D68_metadata.json'

for k in ('∆∆G', 'LE', 'RMSD'):
    metadata[k] = metadata[k].fillna(999).astype(float).round(1)
for k in ('N unconstrained atoms',):
    metadata[k] = metadata[k].fillna(-1).astype(int)
metadata = metadata.copy()


base_url = 'https://www.stats.ox.ac.uk/~ferla/mols/'  # base url path
uuid = '5066835a-a2df-4723-ad92-5adfa622cd74' # uuid of the page target

combo_filename = 'EV-D68_combinations.sdf'
with Chem.SDWriter('upload/'+ combo_filename) as sdw:
    combo_names = []
    for i, row in valids.iterrows():
        if not isinstance(row.merger_mol, Chem.Mol):
            continue
        row.merger_mol.SetProp('_Name', row.merger_name)
        if row.merger_mol.GetNumAtoms() == 0:
            continue
        #AllChem.SanitizeMol(row.merger_mol)
        sdw.write(row.merger_mol)
        # odd way but hey:
        combo_names.append(row['name'])

placement_filename = 'EV-D68_placements.sdf'
with Chem.SDWriter('upload/'+ placement_filename) as sdw:
    placement_names = []
    for i, row in valids.iterrows():
        if not isinstance(row.minimized_mol, Chem.Mol):
            continue
        row.minimized_mol.SetProp('_Name', row['name'])
        if row.minimized_mol.GetNumAtoms() == 0:
            continue
        #AllChem.SanitizeMol(row.minimized_mol)
        sdw.write(row.minimized_mol)
        placement_names.append(row['name'])
        
with open('upload/'+ metadata_filename, 'w') as fh:
    json.dump(dict(
                   headers=headers,
                   data=metadata.values.tolist(),
                   modelnamedex={'combination': combo_names,
                                 'placement': placement_names,
                                },
                   hitnames=hit_names,
            ), fh)
    
# make a page

mike = MikeAPI()
page = mike.get_page(uuid)

page.loadfun = page.get_fragment_js(hit_sdf_url=base_url+hit_filename,
                               model_sdf_urldex={'combination': base_url+combo_filename,
                                                 'placement' : base_url+placement_filename,
                                                },
                               metadata_url=base_url+metadata_filename,
                               model_colordex={'combination': 'salmon',
                                               'placement': 'cyan'},
                               hit_color='grey',
                               template_color='gainsboro',
                               name_col_idx = headers.index('name'),
                               hit_col_idx = headers.index('hits'),
                               target_col_idx = -1, # headers.index('target')
                               sort_col = headers.index('N interactions'),
                               sort_dir = 'desc',
                               fun_name ='loadTable')

# create a way to load the protein
# laziest: 
#page.loadfun += 'setTimeout(loadTable, 1000)'
# better:
page.loadfun += """
window.loadprotein = (prot) => {prot.removeAllRepresentations(); 
                                prot.addRepresentation('cartoon');
                                prot.addRepresentation('line');
                                prot.autoView(); 
                                prot.setName('template');
                                loadTable(); 
                                }
"""
page.loadfun = page.loadfun.replace('"model_colordex": {"combination": "teal", "placement": "teal"}',
                                    '"model_colordex": {"combination": "salmon", "placement": "turquoise"}'
                                   )
page.proteins[0]['loadFx'] = 'loadprotein'
page.title = 'EV-D68 3C protease followups via Fragmenstein'
page.description = f'## Predicted followups\nCombinations in salmon and their purchasable analogue from Enamine Real in turquoise.\n\n Data in table is for make-on-demand analogues. Protein is constant for the sake of memory, but induced fit may have repacked the protein in specific case hence the odd case of calculated nice score and visual clashes.\n\nRMSD cutoff loosened to 2Å'
page.columns_text = 6
page.commit()
print('Done')

In [ ]:
metadata.sort_values('∆∆G')

In [ ]:
print('Done')

In [ ]:
Chem.MolFromSmiles(Chem.MolToSmiles(valids.set_index('name').loc['Z2602896888'].minimized_mol))

In [ ]:
 # ('hbond', 'VAL', 162),
 # ('hbond', 'GLY', 163),
 # ('hbond', 'GLY', 164),


valids.loc[valids[[('halogenbond', 'VAL', 162),
 ('halogenbond', 'GLY', 163),
 ('halogenbond', 'GLY', 164),
 ('halogenbond', 'ASN', 165),
]].sum(axis=1).astype(bool)].sort_values('∆∆G')

In [ ]:
locations = dict(S2=('x1083_0A§1', 'x1305_0B§1', 'x1247_0A§1'),
S1=('x0789_0A§1',
 'x0980_0B§1',
 'x1604_0A§1',
 'x1594_0A§1',
 'x0147_0A§1',
 'x0771_0A§1','x0771benzo', 'x0771_1A§1','x1604base', 'x1604amino', 'x1604hydroxyl',
         'x1498_0A§1','x1498_1B§1', 'x1498_0B§1','x1537_0A§1','x1285§1B',
          'ZINC000000080837', 'ZINC000000168472', 'ZINC000012397068', 'ZINC000015042097', 'ZINC000017061891', 'ZINC000001669068', 'ZINC000001674121', 'ZINC000019779146', 'ZINC000224485041', 'ZINC000238711097', 'ZINC000025924635', 'ZINC000026033116', 'ZINC000002289036', 'ZINC000003852685', 'ZINC000039191805', 'ZINC000045921422', 'ZINC000051951618', 'ZINC000005933792', 'ZINC000066323473', 'ZINC000066347860', 'ZINC000072187487', 'ZINC000072482101', 'ZINC000079065151', 'ZINC000082372119', 'ZINC000082372152', 'ZINC000095830806'
         ),
far_ridge = ('x1285_0B§2', 'x1454_0B§1', 'x1020_0A§1'),
S_1 = ('x0232_0B§3', 'x1453_0B§2', 'x0232_0B§3', 'x0232_1B§3'),
ridge = ('x1052_0A§2', 'x1052_1A§2','x1285_1B§2', 'x1140_0A§1', 'fippedSulfonamide'),
waters = ( 
       'HOH219','HOH240','HOH318','HOH319','HOH322','HOH323','HOH325', 'HOH67', 'trappedHOH', 'oxyanionHOH'),
)

In [ ]:
flipped_locs = {k: l for l, ks in locations.items() for k in ks}
valids['locs'] = valids.hit_names.apply(lambda m: tuple(sorted([flipped_locs[m[0]], flipped_locs[m[1]]])) )
valids = valids.copy()

In [ ]:
valids.locs.value_counts()

In [ ]:
valids.loc[valids['∆∆G'] < -5].sort_values('N_unconstrained_atoms').drop_duplicates('locs')[['name', 'locs', 'smiles', '∆∆G', 'comRMSD','N_constrained_atoms',
                                                  'N_unconstrained_atoms']]

In [ ]:
hit_intxns = pd.read_csv('hit-intxns.csv', index_col=0)

In [ ]:
# change tuple columns to colon separated string columns
valids = valids.rename(columns={col: ':'.join(map(str, col)) for col in valids.columns if isinstance(col, tuple)})
valids = valids.reset_index()

In [ ]:
name2intxns: Dict[str, List[str]] = {n: [xn for xn, xc in xs.items() if xc and xn != 'N_interactions'] for n, xs in hit_intxns.to_dict(orient='index').items()}

In [ ]:
import operator
def tally_hit_intxns(row: pd.Series):
    for hit_name in row.hit_names:
        present_tally = 0
        absent_tally = 0
        for intxn_name in name2intxns[hit_name]:
            if intxn_name not in row.index:
                print(f'missing {intxn_name}')
                absent_tally += 1
            elif row[intxn_name] == 0:
                absent_tally += 1
            else:
                present_tally += 1
    return (present_tally, absent_tally)
                

hit_checks = valids.apply(tally_hit_intxns, axis=1)
valids['N_interactions_kept'] = hit_checks.apply(operator.itemgetter(0))
valids['N_interactions_lost'] = hit_checks.apply(operator.itemgetter(1))

In [ ]:
valids[['name', 'N_interactions', 'N_interactions_kept', 'N_interactions_lost']]

In [ ]:
valids['N_rotatable_bonds'] = valids.minimized_mol.apply(AllChem.CalcNumRotatableBonds)
valids['N_HA'] = valids.minimized_mol.apply(AllChem.CalcNumHeavyAtoms)

In [ ]:
from rdkit import DataStructs
from rdkit.Chem import rdFingerprintGenerator as rdfpg


fpgen = rdfpg.GetRDKitFPGenerator()
hit2fp = {name: fpgen.GetFingerprint(h) for name, h in hits.items()}

def get_similarity(row):
    fp = fpgen.GetFingerprint(AllChem.RemoveHs(row.minimized_mol))
    return max([DataStructs.TanimotoSimilarity(fp,hit2fp[name]) for name in row.hit_names])

valids['max_hit_Tanimoto'] = valids.apply(get_similarity, axis=1)

In [ ]:
for col in ('N_rotatable_bonds', 'max_hit_Tanimoto', 'LE', 'N_interactions', 'N_HA', 'N_interactions_kept', 'N_interactions_lost'):
    px.histogram(valids, col, title=col).show()

In [ ]:
import pandas as pd
from rdkit.Chem import PandasTools, Draw
from rdkit import DataStructs
from rdkit.ML.Cluster import Butina
from rdkit.Chem import rdMolDescriptors as rdmd
from rdkit.Chem import Descriptors
from IPython.display import HTML

def butina_cluster(mol_list,cutoff=0.35):
    # https://github.com/PatWalters/workshop/blob/master/clustering/taylor_butina.ipynb
    fp_list = [rdmd.GetMorganFingerprintAsBitVect(AllChem.RemoveAllHs(m), 3, nBits=2048) for m in mol_list]
    dists = []
    nfps = len(fp_list)
    for i in range(1,nfps):
        sims = DataStructs.BulkTanimotoSimilarity(fp_list[i],fp_list[:i])
        dists.extend([1-x for x in sims])
    mol_clusters = Butina.ClusterData(dists,nfps,cutoff,isDistData=True)
    cluster_id_list = [0]*nfps
    for idx,cluster in enumerate(mol_clusters,1):
        for member in cluster:
            cluster_id_list[member] = idx
    return cluster_id_list

valids['cluster'] = butina_cluster(valids.minimized_mol.to_list())
print(valids['cluster'].max())

In [ ]:
row = valids.set_index('name').loc['Z1694701639']
display2d = lambda mol: display(Chem.MolFromSmiles(Chem.MolToSmiles(AllChem.RemoveHs(mol))))
display2d(row.minimized_mol)
display2d(row.hit_mols[0])
row.max_hit_Tanimoto

In [ ]:
valids['filtered'] = False

In [ ]:
rota_mask = valids['N_rotatable_bonds'] <= 3
LE_mask = valids['LE'] >= 0.2
intxn_mask = valids['N_interactions'] >= 3
ddg_mask = valids['∆∆G'] <= -5
lost_mask = valids['N_interactions_lost'] == 0
no_hit_copies_mask = valids['max_hit_Tanimoto'] <= 0.4
fewer_unconstrained_atoms = valids['N_unconstrained_atoms'] < valids['N_constrained_atoms']

# , ascending=False
filtered = valids.loc[rota_mask & LE_mask & intxn_mask & ddg_mask & lost_mask & no_hit_copies_mask].sort_values('N_interactions', ascending=False)
print(len(filtered))
filtered[['name', '∆∆G', 'LE', 'comRMSD', 'N_rotatable_bonds', 'N_interactions', 'N_HA', 'locs']]

In [ ]:
valids.loc[filtered.drop_duplicates('cluster').index, 'filtered'] = True
valids.filtered.value_counts()

In [ ]:

weights = {'N_rotatable_bonds': 1,
           'LE': -0.2, 
           'N_interactions': -4,
           'N_interactions_lost': +0.1,  # zero would raise error ...
           '∆∆G': 3,
           'N_unconstrained_atoms': 1/3,
           'max_hit_Tanimoto': 0.4
          }

def weight_split(row):
    return [round(row[col] / w, 1) for col, w in weights.items()]

def penalize(row):
    penalty = 0
    for col, w in weights.items():
        penalty += row[col] / w
    return penalty

valids['ad_hoc_penalty'] = valids.apply(penalize, axis=1)
#valids['_split'] = valids.apply(weight_split, axis=1)
f = valids.sort_values('ad_hoc_penalty').drop_duplicates('cluster')
print(len(f))

f.drop_duplicates('locs')[['name', 'hit_names', 'ad_hoc_penalty', 'locs', '∆∆G', 'LE', 'comRMSD', 'N_unconstrained_atoms', 'N_rotatable_bonds', 'LE', 'N_interactions', 'N_interactions_lost', '∆∆G', 'max_hit_Tanimoto', 'N_HA']]

In [ ]:
f = valids.sort_values('ad_hoc_penalty').drop_duplicates('cluster').drop_duplicates('locs')
valids.loc[f.index, 'filtered'] = True
valids.filtered.value_counts()

In [ ]:
valids['has_S1_pocket'] = valids[['hbond:THR:142', 'hbond:HIS:161', 'pistack:HIS:161', 'halogenbond:THR:142']].sum(axis=1)

In [ ]:
# , ('S2', 'ridge')

# PV-002509590804# 

f = valids.loc[valids.locs.isin([('S1', 'S2'), ('S1', 'ridge')]) &
               (valids.has_S1_pocket >= 2)
              ]\
           .sort_values('ad_hoc_penalty')\
           .drop_duplicates('cluster')\
           .head(50)
valids.loc[f.index, 'filtered'] = True
f[['name', 'hit_names', 'ad_hoc_penalty', 'locs', '∆∆G', 'LE', 'comRMSD', 'N_rotatable_bonds', 'LE', 'N_interactions', 'N_interactions_lost', '∆∆G', 'max_hit_Tanimoto', 'N_HA']]

In [ ]:
# , ('S2', 'ridge')
f = valids.loc[valids.locs.isin([('S2', 'ridge')]) & (valids['∆∆G'] <= -4)]\
           .sort_values('ad_hoc_penalty')\
           .drop_duplicates('cluster')\
           .head(10)
valids.loc[f.index, 'filtered'] = True
f[['name', 'hit_names', 'ad_hoc_penalty', 'locs', '∆∆G', 'LE', 'comRMSD', 'N_rotatable_bonds', 'LE', 'N_interactions', 'N_interactions_lost', '∆∆G', 'max_hit_Tanimoto', 'N_HA']]

In [ ]:
valids.filtered.value_counts()

In [ ]:
# worse case
sanity_mask = (valids['N_rotatable_bonds'] <= 6) & \
              (valids['LE'] >= 0.1) & \
              (valids['∆∆G'] <= -3) & \
              (valids['N_interactions'] >= 2) & \
              (valids['N_interactions_lost'] <= 1) & \
              (valids['max_hit_Tanimoto'] <= 0.4) & \
              (valids['N_unconstrained_atoms'] < valids['N_constrained_atoms'])

display(valids.loc[sanity_mask].filtered.value_counts())

from rdkit.Chem import PandasTools

PandasTools.WriteSDF(valids.loc[sanity_mask & valids.filtered],
                     'autoshort-fragmenstein.sdf',
                      molColName='minimized_mol', idName='name',
                     properties=['hit_names', '∆∆G', 'LE', 'comRMSD', 'locs', 'N_HA', 'ad_hoc_penalty', 'N_rotatable_bonds', 'LE', 'N_interactions', 'N_interactions_lost', 'max_hit_Tanimoto'])

In [ ]:
sub = valids.loc[sanity_mask & valids.filtered].reset_index(drop=True)\
            .rename(columns={'smiles': 'SMILES', 
                             'name': 'Vendor_ID',
                             'hit_names': 'Fragment_inspiration_nums',
                            })

sub['Fragalysis_link'] = ''
sub['ZINC_ID'] = 'REAL'
sub['Vendor'] = 'Enamine'
sub['Inventor_surname'] = 'Ferla'
sub['Confidence'] = 1
sub['Method_to_produce'] = 'mergers'
sub['Comments'] = 'Site filtered. Analogue enumerated. Fragmestein mergers. Multiple filters.'
sub['SDF_filename(emailed)'] = 'https://github.com/matteoferla/EV-D68-3C-protease/blob/main/03_merge-fragmenstein/autoshort-fragmenstein.sdf'
sub['Notes_filename(emailed)'] = 'https://github.com/matteoferla/EV-D68-3C-protease/blob/main/03_merge-fragmenstein/fragpipe.ipynb'

sub = sub[['SMILES',
 'Fragment_inspiration_nums',
 'Fragalysis_link',
 'Confidence',
 'Inventor_surname',
 'Method_to_produce',
 'Comments',
 'ZINC_ID',
 'Vendor',
 'Vendor_ID',
 'SDF_filename(emailed)',
 'Notes_filename(emailed)']].to_csv('autoshort-fragmenstein.csv')

In [ ]:
valids.filtered.value_counts()